In [12]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import time

# 1. Загрузка и подготовка данных
df = pd.read_excel('AirQualityUCI.xlsx', parse_dates=[['Date', 'Time']], na_values=-200)
df.rename(columns={'Date_Time': 'timestamp'}, inplace=True)
df.sort_values('timestamp', inplace=True)  # Важная сортировка для временных рядов
df.interpolate(method='linear', limit_direction='both', inplace=True)

# 2. Выбор и нормализация признаков
target = 'CO(GT)'
features = [col for col in df.columns if col not in ['timestamp', target]]

# Правильное разделение данных с учетом временной структуры
train_size = int(0.8 * len(df))
train_df, test_df = df.iloc[:train_size], df.iloc[train_size:]

scaler = StandardScaler()
X_train = scaler.fit_transform(train_df[features])
X_test = scaler.transform(test_df[features])

y_train = train_df[target].values
y_test = test_df[target].values

# 3. Создание временных последовательностей с использованием 24-часовых интервалов
def create_sequences(data, targets, window_size=24):
    X_seq, y_seq = [], []
    for i in range(len(data)-window_size):
        X_seq.append(data[i:i+window_size])
        y_seq.append(targets[i+window_size])
    return np.array(X_seq), np.array(y_seq)

X_train_seq, y_train_seq = create_sequences(X_train, y_train)
X_test_seq, y_test_seq = create_sequences(X_test, y_test)

# 4. Архитектура RNN с LSTM слоями
model = Sequential([
    LSTM(128, input_shape=(X_train_seq.shape[1], X_train_seq.shape[2]), 
        return_sequences=True, dropout=0.2),
    LSTM(64, dropout=0.2),
    Dense(32, activation='relu'),
    Dense(1)
])

model.compile(optimizer='adam',
              loss='mse',
              metrics=['mae', 'mse'])

# 5. Обучение с измерением времени
start_time = time.time()
history = model.fit(X_train_seq, y_train_seq,
                    epochs=100,
                    batch_size=64,
                    validation_split=0.1,
                    shuffle=False,
                    verbose=1)
training_time = time.time() - start_time

# 6. Оценка модели
test_results = model.evaluate(X_test_seq, y_test_seq, verbose=0)
print(f'''
Результаты тестирования:
Время обучения: {training_time:.2f} сек
MAE: {test_results[1]:.4f}
MSE: {test_results[2]:.4f}
''')


C:\Users\User\AppData\Local\Temp\ipykernel_10612\2197748006.py:11: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_excel('AirQualityUCI.xlsx', parse_dates=[['Date', 'Time']], na_values=-200)


Epoch 1/100
105/105 [==============================] - 19s 96ms/step - loss: 1.8480 - mae: 1.0266 - mse: 1.8480 - val_loss: 1.1774 - val_mae: 0.8664 - val_mse: 1.1774
Epoch 2/100
105/105 [==============================] - 9s 89ms/step - loss: 1.1123 - mae: 0.8003 - mse: 1.1123 - val_loss: 0.8442 - val_mae: 0.6861 - val_mse: 0.8442
Epoch 3/100
105/105 [==============================] - 10s 99ms/step - loss: 0.9147 - mae: 0.7061 - mse: 0.9147 - val_loss: 0.7655 - val_mae: 0.6398 - val_mse: 0.7655
Epoch 4/100
105/105 [==============================] - 9s 85ms/step - loss: 0.8625 - mae: 0.6800 - mse: 0.8625 - val_loss: 0.7275 - val_mae: 0.6198 - val_mse: 0.7275
Epoch 5/100
105/105 [==============================] - 9s 82ms/step - loss: 0.8035 - mae: 0.6521 - mse: 0.8035 - val_loss: 0.7109 - val_mae: 0.6039 - val_mse: 0.7109
Epoch 6/100
105/105 [==============================] - 10s 99ms/step - loss: 0.7710 - mae: 0.6433 - mse: 0.7710 - val_loss: 0.7476 - val_mae: 0.6228 - val_mse: 0.7476
E